# A4. Epidemic spreading on complex networks
## Epidemic spreading using the SIS model

Monte Carlo simulation of an epidemic spreading dynamics in complex networks, using the SIS model in which each node represents an individual which can be in two possible states: Susceptible (S), i.e., healthy but can get infected; Infected (I), i.e., has the disease and can spread it to its neighbors.

We are interested in the calculation of the fraction of infected nodes, ρ, in the stationary state, as a function of the infection probability of the disease β (at least 51 values between β=0 and β=1, Δβ=0.02), for different values of the recovery probability μ (e.g. 0.1, 0.5, 0.9). Try different undirected networks (e.g., Erdös-Rényi, scale-free, real), different sizes (at least 500 nodes), average degrees, exponents, etc. Do not make all the combinations, about 10 plots ρ(β) are enough.

Optionally (but mandatory if you want to opt to the maximum grade), compare the ρ(β) result from Monte Carlo simulations with the theoretical prediction provided by the Microscopic Markov Chain Approach (MMCA) model.

### The delivery must include:

Source code

Networks used in Pajek format

Results files

Document (in PDF) including:

Short explanations on how have you done the required work (software, decisions, etc.)
Plots, pointing out “all” the parameters of the corresponding simulation.

Monte Carlo simulations may require a lot of computing time, thus it is recommended the use of compiled languages. In general, and mainly if you are using interpreted languages (e.g., python, Matlab, octave, R), first make tests with small networks, few time steps, and few repetitions, before making the definitive simulations.

Observation: in the original paper on MMCA, the modeled SIS dynamic allows for "one-step reinfections", which are not present in our description of the Monte Carlo simulations. Thus, the last term in Eq. (1) of the paper published in Europhys. Lett. must be removed in order that simulation and theoretical model be comparable. For a comparison of dynamics with and without one-step reinfections, please check the paper published in Int. J. Complex Sys. Sci.


In [ ]:
## install the requirements for the notebook 
!pip install -r requirements.txt

In [71]:
## Imports 
import os 
import igraph as ig
import matplotlib.pylab as plt
from termcolor import colored
import numpy as np 
import random
from tqdm import tqdm
from collections import defaultdict
import gc
import pickle
import itertools
## setting the seeds 
seed1 = np.random.seed(4234)
seed2 = random.seed(4234)

In [72]:
## reloading 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simple SIS Model (NOT FOR THE DELIVERY)

### Defining it 

In [ ]:

## Implementing a simple SIS model with
def sis_infection(susceptible_people:int,       ## Susceptible (S)
                  infected_people:int,          ## Infected    (I)
                  population:int,               ## Population  (N)
                  beta:float,                   ## Transmission Rate prior to change (beta)
                  gamma:float,                  ## Recovery Rate (gamma)
                  n_iterations:int,             ## Number of iterations
                  n_batch:int,                  ## Number of batches
                  verbosity:bool=True,          ## Whether to show every 50th iteration the results 
                  _plot:bool=False) -> list:    ## Whether to plot the final results 
    """
    
    Calculates the SIS model by the following equation: 
    
    INITIAL CONFIGURATION: 
    
    Population  (N) = 100
    Susceptible (S) = N - 1  (100 - 1 = 99)
    Infected    (I) = 1
    
    MODEL: 
    
    For every timeperiod t_{i}, the susceptible and infected populations
    will be recalculated using a set recovery rate ($\gamma$) and a 
    transmission rate prior to change. 
    
    gamma = 0.2 ## recovery rate 
    beta = 0.3  ## transmission rate 
    
    S = (S - beta * S * I/N) + gamma * I
    S = (I - beta * S * I/N) + gamma * I
    
    """
    ## holders 
    sholder = [None] * population
    iholder = [None] * population
    ## text 
    for idx,t in enumerate(range (0, n_iterations)):
        ## update the susceptible population 
        susceptible_people = susceptible_people - (beta * susceptible_people * infected_people / population) + gamma * infected_people
        ## update the infected population 
        infected_people = infected_people + (beta * susceptible_people * infected_people / population) - gamma * infected_people
        ## update the holders 
        sholder[idx] = susceptible_people
        iholder[idx] = infected_people
        ## some feedback
        if verbosity: 
            if t%n_batch == 0:
                it_txt = f"| Iteration = {t} | "
                s_txt = f"Susceptible = {round(susceptible_people,0)} ({susceptible_people/population*100:.2f}%) | "
                i_txt = f"Infected = {round(infected_people,0)} ({infected_people/population*100:.2f}%)|"
                print(it_txt, colored(s_txt, 'green'), colored(i_txt,'red'))
                
    if _plot:
        ## show the plots 
        figure = plt.figure()  ## creating the figure 
        figure.canvas.manager.set_window_title ('SIS model') ## set a title 
        plt.plot(iholder, label='Infected - I(t)') ## plot the I 
        plt.plot(sholder, label='Susceptible - S(t)') ## plot the S 
        plt.legend() ## add legend 
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) ## scientificnotation 
        plt.xlabel('Timepoint (t)') ## labels 
        plt.ylabel('Percentage of Individuals (N in %)') ## labels 
        plt.show()
    ## return the variables 
    return sholder, iholder

### Running it 

In [ ]:
## Initial Population 
N = 1000000
N_ITER = 150
## Number of Susceptible people
S = N - 1
## Number of Infected people 
I = 1
## Parameters 
## Transmission rate prior to change
BETA = 0.5
## Recovery rate
GAMMA = 0.2
## number of batches for feedback 
N_BATCH = 25
## Verbosity 
VERBOSITY = True 
## plotting 
PLOTTING = True
## kwargs
config = {
        "susceptible_people" : S,
        "infected_people"    : I,
        "population"         : N,
        "beta"               : BETA, 
        "gamma"              : GAMMA, 
        "n_iterations"       : N_ITER,
        "n_batch"            : N_BATCH,
        "verbosity"          : VERBOSITY,
        "_plot"              : PLOTTING
        }

## running 
sus, inf = sis_infection(**config)

## Task at hand

### Steps to do: 

0. Implement a simple SIS model [DONE]
1. Perform the SIS modelling on igraph or graph-tools 
2. Implement the Monte Carlo Method for the simulation 
3. Evaluate and average out the results 

#### Read Bibliography Files

In [ ]:
import glob
import pybtex
from pybtex.database.input import bibtex
def load_parse_bib(bib_path:str) -> pybtex.database.BibliographyData:
    """
    Loads the bib files in the directory and returns the given data format
    for the bibliography files. 
    """

    ## start the parser 
    parser = bibtex.Parser()
    ## get the bib files 
    bib_files = glob.glob(bib_path+"/*.bib")
    ## read the files
    bib_data = [parser.parse_file(x) for x in bib_files]
    ## since they are all parsed, we can return the first element 
    ret_bib_data = bib_data[0]
    return ret_bib_data


In [ ]:
from collections import defaultdict
def get_bib_data(path_to_file:str, verbose:bool) -> dict:
    """
    Prints the following information about the citation:
    Title, Journal, Year, Author(s)
    """
    bib_data = load_parse_bib(bib_path = path_to_file)
    holder = defaultdict(dict)
    ## article
    papers = list(bib_data.entries.keys())
    ## info 
    for idx,ids in enumerate(papers):
        c_data = bib_data.entries[ids]
        title = c_data.fields['title']
        journal = c_data.fields['journal']
        year = c_data.fields['year']
        ## authors 
        authors = [" ".join(x.first_names + x.last_names)  for x in c_data.persons['author']][:3]
        ## joined on a string 
        ret_authors = ' '.join((x for x in authors))
        ## update tracker
        holder[idx]['title'] = title
        holder[idx]['journal'] = journal
        holder[idx]['year'] = year
        holder[idx]['author'] = authors[0]
        if verbose:
            print(colored(f'Source {idx+1}','green'))
            print(colored("Title:",attrs=['bold']),f"{title}")
            print(colored("Journal:",attrs=['bold']),f"{journal}")
            print(colored("Year:",attrs=['bold']),f"{year}")
            print(colored("Authors:",attrs=['bold']),f"{ret_authors}\n")
    return holder

In [ ]:
## where the citation (.bib) files are located
CITATIONS = "./citations"
VERBOSITY = True
## returns a dictionary 
info = get_bib_data(path_to_file=CITATIONS,verbose=VERBOSITY)

### SIS Model

## Introduction: Epidemiological Models

Epidemic modelling is a field which has come under a lot of attention ever since the onset of COVID-19, at a much larger scale than before. In order to depict the spread of the disease, it is necessary 

The Susceptible (S) Infected (I) Susceptible (S) (SIS) Model is a used

##### SIS SETUP PARAMETERS

In [93]:
from src.config import main_config
from src.MC import MC_SIS
from src.graph_config import make_graph_config

## the initial 
MC_SIS_CONFIG, N_REPS, COMBS = main_config()
GRAPHS = make_graph_config(save=True, out_dir = './Networks')

results = MC_SIS(
                graph_dict=GRAPHS,
                mu_beta_comb = COMBS,
                n_reps = N_REPS,
                params = MC_SIS_CONFIG
                        )

| Processing | ER_900_0.6 | Repetition: 0 | Mu: 0.1 | Beta: 0.0 |	:   0%|          | 0/4 [00:00<?, ?it/s]

In [60]:
## guardar todos los resultados 
with open('./out/all_graph_results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'res' is not defined

In [ ]:
## look at the keysof the dictionary
print(results.keys()) ## these should match the networks & have each mu as a key 

In [ ]:
## math rho/beta symbol 
rho = r"$\rho$"
beta = r"$\beta$"
## plot rho against beta
for graph in results.keys():
    for mu0 in graph.keys():
        rhos = [x for x in results[graph][mu0].values()]
        betas = [x for x in results[graph][mu0].keys()]
        plt.plot(betas,rhos, label=f"$\mu$={mu0}",marker="o")
    ## set the title 
    ## mathematical symbol for rho 
    plt.title(f"Experimental Rho ({rho}) vs Beta ({beta}) - {graph}")
    plt.xlabel(f"{beta}")
    plt.ylabel(f"{rho}")
    plt.legend()
plt.show()

In [ ]:
## math rho/beta symbol 
rho = r"$\rho$"
beta = r"$\beta$"
## plot rho against beta for each of the keys of the res dictionary
N_PLOTS = len(res.keys())
N_COLS = 2 ## number of columns
N_ROWS = int(np.ceil(N_PLOTS/N_COLS)) ## number of rows
## now iterate through the graphs and plot the rho against the beta for each graph as a title 
fig, axs = plt.subplots(nrows=N_ROWS, ncols=N_COLS, figsize=(10,10))
axs = axs.ravel()
for idx, graph in enumerate(res.keys()):
    for mu0 in graph.keys():
        rhos = [x for x in res[graph][mu0].values()]
        betas = [x for x in res[graph][mu0].keys()]
        axs[idx].plot(betas,rhos, label=f"$\mu$={mu0}",marker="o")
    axs[idx].set_title(f"Experimental Rho ({rho}) vs Beta ({beta}) - {graph}")
    axs[idx].set_xlabel(f"{beta}")
    axs[idx].set_ylabel(f"{rho}")
    axs[idx].legend()
plt.title("Experimental Rho vs Beta")
plt.tight_layout()
plt.show()